In [1]:
import pennylane as qml
import numpy as np
import cirq
#use pytorch for the OMD implementation
import torch as torch
#optimistic Adam class in optim file
import optim

Any computational object that can apply quantum operations and return a measurement value is called a quantum device.

In PennyLane, a device could be a hardware device (such as the IBM QX4, via the PennyLane-PQ plugin), or a software simulator (such as Strawberry Fields, via the PennyLane-SF plugin).


In [2]:
dev = qml.device('cirq.simulator', wires=1)

Unitary = arbitrary rotation


In [3]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
angles = [phi,theta,omega]

def real(angles, **kwargs):
    qml.Rot(*angles, wires=0)
        

Generator

In [4]:
def generator(w, **kwargs):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)

Discriminator


In [5]:
def discriminator(w):
    qml.RZ(w[0], wires=0)
    qml.RY(w[1], wires=0)
    qml.RZ(w[2], wires=0)


2 nodes:  1 for data-discriminator
          1 for generator-discriminator
          

In [6]:
@qml.qnode(dev, interface="torch", diff_method = 'parameter-shift')
def real_disc_circuit(angles, disc_weights):
    real(angles)
    discriminator(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))

@qml.qnode(dev, interface="torch", diff_method = 'parameter-shift')
def gen_disc_circuit(gen_weights, disc_weights):
    generator(gen_weights)
    discriminator(disc_weights[0:3])
    return qml.expval(qml.PauliZ(0))


Cost Functions

In [7]:
def prob_real_true(disc_weights):
    true_disc_output = real_disc_circuit(angles, disc_weights)
    # convert to probability
    prob_real_true = (true_disc_output + 1) / 2
    return prob_real_true


def prob_fake_true(gen_weights, disc_weights):
    fake_disc_output = gen_disc_circuit(gen_weights, disc_weights)
    # convert to probability
    prob_fake_true = (fake_disc_output + 1) / 2
    return prob_fake_true


def disc_cost(disc_weights, gen_weights):
    cost = prob_fake_true(gen_weights, disc_weights) - prob_real_true(disc_weights)
    return cost


def gen_cost(gen_weights, disc_weights):
    return -prob_fake_true(gen_weights, disc_weights)

initial weights

In [8]:
phi = np.pi / 6
theta = np.pi / 2
omega = np.pi / 7
np.random.seed(0)
eps = 1e-2
init_gen_weights = np.array([np.pi] + [0] * 2) + \
                   np.random.normal(scale=eps, size=(3,))
init_disc_weights = np.random.normal(size=(3,))

gen_weights_grad  = torch.tensor(init_gen_weights, requires_grad=True)
disc_weights_grad = torch.tensor(init_disc_weights, requires_grad=True)

Training

In [9]:
N = 100      #Number of training cycles
steps = 20 #Number of steps for discriminator training
ratio = 1/20  #disc/gen training ratio

for i in range (N):
    
    #training discriminator
    #OAdam only updates the parameters with requires_grad = True and doesn't change the other ones
    gen_weights_non_grad = torch.tensor(gen_weights_grad.detach().numpy(), requires_grad = False)
    #need an optimizer for both cases seperately
    opt = optim.OAdam([disc_weights_grad, gen_weights_non_grad], lr = 0.1)
    
    #los function
    def closure():
        opt.zero_grad()
        loss = disc_cost(disc_weights_grad, gen_weights_non_grad)
        loss.backward()
        return loss

    #do the discriminator training steps
    for j in range(steps):
        #use  step and not minimize
        opt.step(closure)
        if j % 5 == 0:
            cost_val = disc_cost(disc_weights_grad, gen_weights_non_grad).detach().numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class real as real: {}'.format(j, steps, cost_val, prob_real_true(disc_weights_grad).detach().numpy()))

    
    print('discriminator trained')
    
    #training generator
    #switch requires_grad for both weight sets
    disc_weights_non_grad = torch.tensor(disc_weights_grad.detach().numpy(), requires_grad = False)
    gen_weights_grad = torch.tensor(gen_weights_non_grad.detach().numpy(), requires_grad = True)
    #define new optimizer
    opt = optim.OAdam([disc_weights_non_grad, gen_weights_grad], lr = 0.1)
    
    #generator loss
    def closure():
        opt.zero_grad()
        loss = gen_cost(gen_weights_grad, disc_weights_non_grad)
        loss.backward()
        return loss

    #do the optimization with steps*ratio
    for k in range(int(steps*ratio)):
        opt.step(closure)
        if k % 5 == 0:
            cost_val = gen_cost(gen_weights_grad, disc_weights_non_grad).detach().numpy()
            print('Epoch  {}/{}, Cost: {}, Probability class fake as real: {}'.format(k, steps*ratio, cost_val, prob_fake_true(gen_weights_grad, disc_weights_non_grad).detach().numpy()))

    
    #switch discriminator weights back to requires_grad
    disc_weights_grad = torch.tensor(disc_weights_non_grad.detach().numpy(), requires_grad = True)
    print('end of cycle', i)
    print('\n')


C:\Users\sarah\Downloads\optim.py:95: UserWarning: This overload of addcdiv_ is deprecated:
	addcdiv_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcdiv_(Tensor tensor1, Tensor tensor2, *, Number value) (Triggered internally at  ..\torch\csrc\utils\python_arg_parser.cpp:1050.)
  p.data.addcdiv_(step_size, exp_avg, exp_avg_sq.sqrt().add(group['eps']))


Epoch  0/20, Cost: -0.6626259014010429, Probability class real as real: 0.9256928488612175
Epoch  5/20, Cost: -0.7001638375222683, Probability class real as real: 0.8127418607473373
Epoch  10/20, Cost: -0.6998931020498276, Probability class real as real: 0.812340259552002
Epoch  15/20, Cost: -0.7057937160134315, Probability class real as real: 0.8553089052438736
discriminator trained
Epoch  0/1.0, Cost: -0.24021577090024948, Probability class fake as real: 0.24021577090024948
end of cycle 0


Epoch  0/20, Cost: -0.6285642758011818, Probability class real as real: 0.7848358079791069
Epoch  5/20, Cost: -0.633650928735733, Probability class real as real: 0.820900022983551
Epoch  10/20, Cost: -0.6337005198001862, Probability class real as real: 0.8159846141934395
Epoch  15/20, Cost: -0.6337077766656876, Probability class real as real: 0.8169958516955376
discriminator trained
Epoch  0/1.0, Cost: -0.26904481649398804, Probability class fake as real: 0.26904481649398804
end of cycle 1


Epoch

discriminator trained
Epoch  0/1.0, Cost: -0.7145194262266159, Probability class fake as real: 0.7145194262266159
end of cycle 16


Epoch  0/20, Cost: 0.08201078325510025, Probability class real as real: 0.7129944860935211
Epoch  5/20, Cost: 0.0013702288269996643, Probability class real as real: 0.8999555222690105
Epoch  10/20, Cost: -0.06890270113945007, Probability class real as real: 0.8308626264333725
Epoch  15/20, Cost: -0.09299905598163605, Probability class real as real: 0.6451859325170517
discriminator trained
Epoch  0/1.0, Cost: -0.5893710404634476, Probability class fake as real: 0.5893710404634476
end of cycle 17


Epoch  0/20, Cost: 0.016508713364601135, Probability class real as real: 0.4958193600177765
Epoch  5/20, Cost: 0.006560534238815308, Probability class real as real: 0.3123433291912079
Epoch  10/20, Cost: -0.009045198559761047, Probability class real as real: 0.376486673951149
Epoch  15/20, Cost: -0.020269975066184998, Probability class real as real: 0.566604465246

Epoch  5/20, Cost: 0.0017834454774856567, Probability class real as real: 0.9006207101047039
Epoch  10/20, Cost: -0.0687011182308197, Probability class real as real: 0.8325096070766449
Epoch  15/20, Cost: -0.09306538105010986, Probability class real as real: 0.6465608030557632
discriminator trained
Epoch  0/1.0, Cost: -0.59006467461586, Probability class fake as real: 0.59006467461586
end of cycle 33


Epoch  0/20, Cost: 0.016528472304344177, Probability class real as real: 0.4967096596956253
Epoch  5/20, Cost: 0.006632328033447266, Probability class real as real: 0.31219837069511414
Epoch  10/20, Cost: -0.008973151445388794, Probability class real as real: 0.37483273446559906
Epoch  15/20, Cost: -0.02024441957473755, Probability class real as real: 0.5653569400310516
discriminator trained
Epoch  0/1.0, Cost: -0.7145242542028427, Probability class fake as real: 0.7145242542028427
end of cycle 34


Epoch  0/20, Cost: 0.08231403678655624, Probability class real as real: 0.712802708148956

Epoch  15/20, Cost: -0.09306538105010986, Probability class real as real: 0.646563783288002
discriminator trained
Epoch  0/1.0, Cost: -0.5900689214468002, Probability class fake as real: 0.5900689214468002
end of cycle 49


Epoch  0/20, Cost: 0.016529321670532227, Probability class real as real: 0.496714323759079
Epoch  5/20, Cost: 0.006634369492530823, Probability class real as real: 0.31219831109046936
Epoch  10/20, Cost: -0.008970022201538086, Probability class real as real: 0.37482529878616333
Epoch  15/20, Cost: -0.020241573452949524, Probability class real as real: 0.5653521865606308
discriminator trained
Epoch  0/1.0, Cost: -0.7145287841558456, Probability class fake as real: 0.7145287841558456
end of cycle 50


Epoch  0/20, Cost: 0.0823163390159607, Probability class real as real: 0.7128051072359085
Epoch  5/20, Cost: 0.0017857104539871216, Probability class real as real: 0.9006220325827599
Epoch  10/20, Cost: -0.06870041787624359, Probability class real as real: 0.832512505352

Epoch  5/20, Cost: 0.006636574864387512, Probability class real as real: 0.31219664216041565
Epoch  10/20, Cost: -0.008966699242591858, Probability class real as real: 0.37481309473514557
Epoch  15/20, Cost: -0.02023857831954956, Probability class real as real: 0.5653429627418518
discriminator trained
Epoch  0/1.0, Cost: -0.7145310342311859, Probability class fake as real: 0.7145310342311859
end of cycle 66


Epoch  0/20, Cost: 0.08231887966394424, Probability class real as real: 0.7128049433231354
Epoch  5/20, Cost: 0.001787666231393814, Probability class real as real: 0.9006230942904949
Epoch  10/20, Cost: -0.06869979947805405, Probability class real as real: 0.8325148671865463
Epoch  15/20, Cost: -0.0930653065443039, Probability class real as real: 0.6465668231248856
discriminator trained
Epoch  0/1.0, Cost: -0.590073898434639, Probability class fake as real: 0.590073898434639
end of cycle 67


Epoch  0/20, Cost: 0.016530349850654602, Probability class real as real: 0.49671976268291

Epoch  15/20, Cost: -0.020235583186149597, Probability class real as real: 0.5653372555971146
discriminator trained
Epoch  0/1.0, Cost: -0.7145346254110336, Probability class fake as real: 0.7145346254110336
end of cycle 82


Epoch  0/20, Cost: 0.08232142776250839, Probability class real as real: 0.7128061801195145
Epoch  5/20, Cost: 0.0017894245684146881, Probability class real as real: 0.9006249979138374
Epoch  10/20, Cost: -0.06869915127754211, Probability class real as real: 0.8325173333287239
Epoch  15/20, Cost: -0.09306536614894867, Probability class real as real: 0.6465697437524796
discriminator trained
Epoch  0/1.0, Cost: -0.5900783240795135, Probability class fake as real: 0.5900783240795135
end of cycle 83


Epoch  0/20, Cost: 0.01653110980987549, Probability class real as real: 0.49672508239746094
Epoch  5/20, Cost: 0.006638914346694946, Probability class real as real: 0.31219616532325745
Epoch  10/20, Cost: -0.00896318256855011, Probability class real as real: 0.37480381131

Epoch  5/20, Cost: 0.0017916113138198853, Probability class real as real: 0.9006256125867367
Epoch  10/20, Cost: -0.06869864463806152, Probability class real as real: 0.8325202763080597
Epoch  15/20, Cost: -0.09306532144546509, Probability class real as real: 0.6465730369091034
discriminator trained
Epoch  0/1.0, Cost: -0.5900827795267105, Probability class fake as real: 0.5900827795267105
end of cycle 99




Bloch representation & difference in the vectors

In [10]:
obs = [qml.PauliX(0), qml.PauliY(0), qml.PauliZ(0)]

bloch_vector_real = qml.map(real, obs, dev, interface="torch")
bloch_vector_generator = qml.map(generator, obs, dev, interface="torch")

bloch_real = bloch_vector_real([phi, theta, omega]).detach().numpy()
bloch_generated = bloch_vector_generator(gen_weights_grad).detach().numpy()
difference = np.absolute(bloch_real - bloch_generated)

print("Real Bloch vector: {}".format(bloch_real))
print("Generator Bloch vector: {}".format(bloch_generated))
print("Difference: {}".format(difference))



Real Bloch vector: [0.90096882 0.43388376 0.        ]
Generator Bloch vector: [ 0.91672719  0.39813149 -0.03320512]
Difference: [0.01575837 0.03575227 0.03320512]
